# Step-by-step walkthrough of Etch contract development and deployment

### **Installation (for first-time use)**

Install Anaconda (if not done already): [Download Link](<https://docs.anaconda.com/anaconda/install/>)

Setup Python env "etch" for Etch contract development. It is currently tested for Python3.6.

```
conda update conda -y
conda create -n etchenv python=3.6 -y
conda activate etchenv
```

Pip install Ledger Python API. The latest version is currently alpha v0.10.x. [Resource](https://docs.fetch.ai/getting-started/python-api-install/)

`pip install -U fetchai-ledger-api`

Alternatively you can install directly from source:

```
git clone https://github.com/fetchai/ledger-api-py.git
cd ledger-api-py
pip uninstall fetchai-ledger-api
```

Add etchenv as an iPython kernel and restart Jupyter Notebook. That way you can see "etchenv" as a kernel option.

`python -m ipykernel install --user --name etchenv --display-name "etch"`

There are two ways settting up your Etch Ledger locally:
    1) From Source
    2) From Docker

***Setting up Ledger from Source***

Install the Fetch ledger at your workspace directory. See see Fetch [docs](https://docs.fetch.ai/) "Getting Started" section for instructions to do so. Currently, it is compatible with `MacOS`, `Ubuntu`, and `Redhat` systems.

Once built (just `make -j 4 constellation`), move to the following directory:

`cd ledger/build/apps/constellation`

Run the Fetch ledger on your local testnode with the following command:

`./constellation -port 8100 -block-interval 3000 -standalone`

The output will show the following in console:

```
F E ╱     Constellation v0.4.1-rc1
   T C     Copyright 2018-2019 (c) Fetch AI Ltd.
     H

[ 2019-06-03 16:55:20.215, # 1 INFO  :                                main ] Configuration:

port......................: 8100
network mode..............: Standalone
num executors.............: 1
num lanes.................: 1
num slices................: 500
bootstrap.................: 0
discoverable..............: 0
host name.................:
external address..........: 127.0.0.1
db-prefix.................: node_storage
interface.................: 127.0.0.1
mining....................: Yes
tx processor threads......: 12
shard verification threads: 12
block interval............: 3000ms
max peers.................: 3
peers update cycle........: 0ms
peers.....................:
manifest.......:
 - HTTP/0: tcp://127.0.0.1:8100 (8100)
 - CORE/0: tcp://127.0.0.1:8101 (8101)
 - Lane/0: tcp://127.0.0.1:8110 (8110)
```

***Setting up Ledger from Docker***

Install Docker CE engine locally (if you haven't already). [Link](https://docs.docker.com/install/linux/docker-ce/ubuntu/)

Pull latest Constellation image from Docker Hub. [Link](https://hub.docker.com/r/fetchai/constellation)

`docker pull fetchai/constellation`

Run the Fetch Ledger via Docker. [Link](https://github.com/fetchai/docker-constellation/blob/master/docs/description.md)

`docker run -p 8100:8100 fetchai/constellation -port 8100 -block-interval 3000 -standalone`

The output should appear in console:

```
 F E ╱     Constellation v0.9.1
   T C     Copyright 2018-2019 (c) Fetch AI Ltd.
     H     

[I] 2019/11/27 16:19:14 | main                           : Input Configuration:
lanes................: 1
slices...............: 500
block-interval.......: 3000
standalone...........: Yes
private-network......: No
db-prefix............: node_storage
port.................: 8100
peers................: 
external.............: 127.0.0.1
config...............: 
max-peers............: 3
transient-peers......: 1
peers-update-cycle-ms: 0
disable-signing......: No
kademlia-routing.....: Yes
bootstrap............: No
discoverable.........: No
host-name............: 
network..............: 
token................: 
processor-threads....: 4
verifier-threads.....: 4
executors............: 1
load-genesis-file....: No
genesis-file-location: 
experimental.........: 
pos..................: No
max-cabinet-size.....: 10
stake-delay-period...: 5
aeon-period..........: 100
```

The following walkthrough should work regardless of whether the Ledger is built from source or from Docker.

### **Etch Development Walkthrough**

#### **Step 1) Setup Python API and initialize contract owner entity**

Print the Ledger Python API version. Make sure that it is the latest. 

In [370]:
from fetchai.ledger import __version__ as version_string
print(version_string)

0.10.0-a6


Import the following to access Ledger Python API

In [371]:
from fetchai.ledger.api import LedgerApi
from fetchai.ledger.contract import Contract
from fetchai.ledger.crypto import Entity, Address

Set up Python API, using your local config: 

In [372]:
# Constellation config
HOST = '127.0.0.1'
PORT = 8100

Create an API instance

In [373]:
api = LedgerApi(HOST, PORT)

Create an Entity for owner of the Etch contract, where contains owner's public/private keypair as well as its wallet address. In the pet-shop example, the owner can be the shelter iteself.

In [374]:
# Create keypair for the contract owner
owner = Entity()
owner_addr = Address(owner)

In [375]:
print(owner_addr)

xEeapmwLmsfMACtvuuLSBUWz9oMcwFxxMrMK9mWhdUZqYtXct


(Optional) Save the contract owner's private key to disk with a password

In [376]:
with open('owner_private.key', 'w') as private_key_file:
    owner.dump(private_key_file, password="pass")

#### **Step 2a) Deploy the hello-world contract**

Read-in script from hello.etch

In [34]:
source = "hello-world/hello.etch"

with open(source, 'r') as file:
    contract_text = file.read()

In [35]:
print(contract_text)

//------------------------------------------------------------------------------
//
//   Copyright 2019 Fetch.AI Limited
//
//   Licensed under the Apache License, Version 2.0 (the "License");
//   you may not use this file except in compliance with the License.
//   You may obtain a copy of the License at
//
//       http://www.apache.org/licenses/LICENSE-2.0
//
//   Unless required by applicable law or agreed to in writing, software
//   distributed under the License is distributed on an "AS IS" BASIS,
//   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
//   See the License for the specific language governing permissions and
//   limitations under the License.
//
//------------------------------------------------------------------------------

// This contract require release v0.10.1 alpha or above
// Note that this is experimental functionality


@init
function createMessage(owner : Address)

    var name : String = "world";
    var state = State<String>("g

***TO-DO***: writeup about the contract desc

Create contract object from contract text:

In [36]:
contract = Contract(contract_text, owner)

Top-up 20,000 FET tokens to owner entity in order to pay for ledger tx fees. Note: this is your local testnet so you can print new tokens at-will.

In [37]:
api.sync(api.tokens.wealth(owner, 20000))

Print the owner's balance to confirm the 20,000 has been received

In [38]:
print(api.tokens.balance(address=owner_addr))

71506


Deploy the contract on-chain, paying 10,000 in transaction fees (an arbitrary amount).

In [39]:
api.sync(api.contracts.create(owner, contract, 10000))

Alternatively Execute `python deploy.py pet` to deploy pet-shop contract.

#### **Step 2b) Interact with the hello-world contract**

Once the contract has been successfully deployed, we can first test it by querying `persistentGreeting`.

In [40]:
# Printing message
print(contract.query(api, 'persistentGreeting'))

Hello, world!


Now, let's change the greeting. First, set the transfer fee amount:

In [41]:
# Initialize tx fee
tok_transfer_amount = 200
fet_tx_fee = 100

Then change the greeting recipient:

In [42]:
# Execute smart contract5
recipient = "Earth"
result = api.sync(contract.action(api, 'changeGreeting', fet_tx_fee, [owner], recipient))

Print the greeting again. Now you see that it has been changed.

In [43]:
# Printing message
print(contract.query(api, 'persistentGreeting'))

Hello, Earth!


#### **Step 3a) Deploy the pet-shop contract**

Read-in script from adoption.etch

In [364]:
source = "pet-shop/adoption_map.etch"

with open(source, 'r') as file:
    contract_text = file.read()

Print adoption contract text:

In [365]:
print(contract_text)

//------------------------------------------------------------------------------
//
//   Copyright 2019 Fetch.AI Limited
//
//   Licensed under the Apache License, Version 2.0 (the "License");
//   you may not use this file except in compliance with the License.
//   You may obtain a copy of the License at
//
//       http://www.apache.org/licenses/LICENSE-2.0
//
//   Unless required by applicable law or agreed to in writing, software
//   distributed under the License is distributed on an "AS IS" BASIS,
//   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
//   See the License for the specific language governing permissions and
//   limitations under the License.
//
//------------------------------------------------------------------------------

// This contract require release v0.10.1 alpha or above
// Note that this is experimental functionality

persistent adopters : Map<Int32, Address>;


@init
function constructor(owner : Address)

    use adopters;
    a

***TO-DO***: writeup about the contract desc

Create contract object from contract text:

In [366]:
contract = Contract(contract_text, owner)

#get contract [digest, address]
contract.name.split('.')

['4aa32c6c886a410a6c8241af18dda5f5fb9268dd9e1807ff471bfaaa6c636695',
 '2vvL1PM4Y7q1zr2GC3hQUEFTVZNn572iHL3H5hrgGUU2bgePsj']

Top-up 20,000 FET tokens to owner entity in order to pay for ledger tx fees. Note: this is your local testnet so you can print new tokens at-will.

In [356]:
api.sync(api.tokens.wealth(owner, 20000))

Print the owner's balance to confirm the 20,000 has been received

In [367]:
print(api.tokens.balance(address=owner_addr))

26446


Deploy the contract on-chain, paying 10,000 in transaction fees (an arbitrary amount).

In [368]:
api.sync(api.contracts.create(owner, contract, 10000))

Alternatively Execute `python deploy.py pet` to deploy pet-shop contract.

#### **Step 3b) Interact with the pet-shop contract**

Create keypair for new entity that interacts with contract. In this case, this will be adopter looking to adopt a pet from the shelter.

In [144]:
adopter1 = Entity()
adopter_addr1 = Address(adopter1)

In [145]:
print(adopter_addr1)

2w63GpakWFkPk7uYzdbjYP4cLGMYZvRadKNAHtD3DG5y8ds5Lj


Top-up the adopter entity as well as the user also needs FET tokens to pyay for tx fees when interacting the the contract.

In [146]:
api.sync(api.tokens.wealth(adopter1, 10000))

Print adopter1's balance to confirm.

In [147]:
print(api.tokens.balance(address=adopter1))

10000


First, query *getAdopters* function to see which pets are adopted.

In [369]:
# Printing message
result = contract.query(api, 'getAdopters')

RuntimeError: Failed to make requested query with no error message.

In [90]:
print(result)

h863r8WYUE7pfHxASCzwapPD9YD2DWuAo58nA4CyAhK3AwXTF


Next, pick a pet to adopt. Let's say pet on index 10.

In [319]:
# Initialize tx fee
tok_transfer_amount = 200
fet_tx_fee = 100

In [320]:
# Execute smart contract5
pet_id = 10
result = api.sync(contract.action(api, 'adopt', fet_tx_fee, [owner], pet_id))

In [321]:
print(result)

In [322]:
result = contract.query(api, 'getAdopterOfPet', petId=pet_id)

In [323]:
print(result)

2K4ZLsVXj1ifxZS2HE7jsaK7KwoxXVFuJC1aCpQqrUkpWL3SWs


In [325]:
result = contract.query(api, 'getAdopters')
print(result)

None


In [159]:
# Execute smart contract5
pet_id = 2
result = api.sync(contract.action(api, 'adopt', fet_tx_fee, [owner], pet_id))

RuntimeError: Some transactions have failed: 7af5a64a99c58783ef1e0e6d515b8bde31b4e40e262e9668c9216bbedf38e25b:Insufficient charge

Query *getAdopters* to confirm that pet on index 10 has been adopted by adopter1, with adopter1's address listed.

In [ ]:
# Printing message
print(contract.query(api, 'getAdopters'))

Let's create anther adopter.

In [ ]:
adopter2 = Entity()
adopter_addr2 = Address(adopter2)

In [ ]:
print(adopter_addr2)

In [ ]:
# Top-up the adopter
api.sync(api.tokens.wealth(adopter2, 10000))

Adopter2 will first try to pet on index 10. However, that execution should fail as the pet is already have taken by Adopter1.

In [ ]:
pet_id = 10
result = api.sync(contract.action(api, 'adopt', fet_tx_fee, [adopter1], pet_id))

Thus, Adopter2 will settle with the pet on index 5 instead:

In [ ]:
pet_id = 5
result = api.sync(contract.action(api, 'adopt', fet_tx_fee, [adopter1], pet_id))

Let's take a look at *getAdopters* one last time:

In [ ]:
# Printing message
print(contract.query(api, 'getAdopters'))

#### **Step 4a) Deploy the simple-open-auction contract**

Now let's try to deploy a more challenging auction contract.

In [377]:
source = "simple-open-auction/auction.etch"

with open(source, 'r') as file:
    contract_text = file.read()

Print auction contract text:

In [378]:
print(contract_text)

 //------------------------------------------------------------------------------
//
//   Copyright 2019 Fetch.AI Limited
//
//   Licensed under the Apache License, Version 2.0 (the "License");
//   you may not use this file except in compliance with the License.
//   You may obtain a copy of the License at
//
//       http://www.apache.org/licenses/LICENSE-2.0
//
//   Unless required by applicable law or agreed to in writing, software
//   distributed under the License is distributed on an "AS IS" BASIS,
//   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
//   See the License for the specific language governing permissions and
//   limitations under the License.
//
//------------------------------------------------------------------------------

// This contract require release v0.9.1 or above
// Note that this is experimental functionality

persistent beneficiary : Address;
persistent auctionEndTime : UInt64;
persistent highestBidder : Address;
persistent hi

***TO-DO***: writeup about the contract desc

Create contract object from contract text:

In [379]:
contract = Contract(contract_text, owner)

Top-up 20,000 FET tokens to owner entity in order to pay for ledger tx fees. Note: this is your local testnet so you can print new tokens at-will.

In [380]:
api.sync(api.tokens.wealth(owner, 20000))

Print the owner's balance to confirm the 20,000 has been received

In [381]:
print(api.tokens.balance(address=owner_addr))

20000


Deploy the contract on-chain, paying 10,000 in transaction fees (an arbitrary amount).

In [382]:
api.sync(api.contracts.create(owner, contract, 10000))

RuntimeError: Some transactions have failed: 8e7ffcbbd7eba45386633b34b4efe0caf7a25e6283c733cfad29df23015aef3b:Contract Execution Failure

Check the current block number:

Alternatively Execute `python deploy.py pet` to deploy pet-shop contract.

#### **Step 4b) Interact with the simple-open-auction contract**

Create keypair for new entity that interacts with contract. In this case, this will be adopter looking to adopt a pet from the shelter.

In [ ]:
bidder1 = Entity()
bidder_addr1 = Address(adopter1)

print(bidder1)

Top-up the adopter entity as well as the user also needs FET tokens to pyay for tx fees when interacting the the contract.

In [ ]:
api.sync(api.tokens.wealth(bidder1, 10000))

First, query *getAdopters* function to see which pets are adopted.

In [ ]:
# Printing message
print(contract.query(api, 'getAdopters'))

Next, pick a pet to adopt. Let's say pet on index 10.

In [ ]:
# Initialize tx fee
tok_transfer_amount = 200
fet_tx_fee = 100

make a bid amount:

In [ ]:
bid_amount = 2000 
api.sync(contract.action(api, 'bid', fet_tx_fee, bid_amount)